# Семинар 5.
## Линейная алгебра и нейросети

## Нейросеть как суперпозиция функций

- Линейных и нелинейных
- Можно представить в виде графа вычислений
- С помощью этого графа можно [автоматически вычислять градиенты](https://en.wikipedia.org/wiki/Automatic_differentiation)

## Что такое градиент и как его вычислить?

- Если функция $f$ зависит от вектора или матрицы и вычисляет скаляр, то её градиент по аргументу - это вектор или матрица, элементы которых – производные выхода по соответствующему элементу аргумента
- Пример: $f(x) = x^{\top}Ax + b^{\top}x$. Чему равен градиент?
- Поэлементный пример: $f(x) = x^2$ поэлементно. Чему равен градиент (матрица Якоби)?

In [1]:
import torch

n = 5
A = torch.randn((n, n), requires_grad=True)
b = torch.randn((n,))
x = torch.randn((n,), requires_grad=True)

f = 0.5 * x @ A @ x - b @ x
f.backward()
print(f)
print(f.item())

tensor(-0.3647, grad_fn=<SubBackward0>)
-0.3646929860115051


In [2]:
manual_grad_x = 0.5 * (A + A.t()) @ x - b

print(manual_grad_x.data)
print(x.grad.data)

tensor([ 0.0815, -2.8794,  0.9538,  0.2398, -1.8685])
tensor([ 0.0815, -2.8794,  0.9538,  0.2398, -1.8685])


In [3]:
manual_grad_A = 0.5 * torch.ger(x, x)

print(manual_grad_A.data)
print(A.grad.data)
print(torch.norm(manual_grad_A.data - A.grad.data).item())

tensor([[0.8848, 0.1198, 0.1101, 0.0458, 0.1378],
        [0.1198, 0.0162, 0.0149, 0.0062, 0.0186],
        [0.1101, 0.0149, 0.0137, 0.0057, 0.0171],
        [0.0458, 0.0062, 0.0057, 0.0024, 0.0071],
        [0.1378, 0.0186, 0.0171, 0.0071, 0.0215]])
tensor([[0.8848, 0.1198, 0.1101, 0.0458, 0.1378],
        [0.1198, 0.0162, 0.0149, 0.0062, 0.0186],
        [0.1101, 0.0149, 0.0137, 0.0057, 0.0171],
        [0.0458, 0.0062, 0.0057, 0.0024, 0.0071],
        [0.1378, 0.0186, 0.0171, 0.0071, 0.0215]])
0.0


## Причём тут нейросети?

- Обучение с учителем (но не только!)
- Есть данные, есть ответы
- Нейросеть $\approx$ сложная композиция простых функций, которая по данным восстанавливает ответы
- Для обучения необходима целевая функция a.k.a. функция потерь или loss, который минимизируется каким-нибудь стохастическим методом первого порядка
- Поэтому нужны градиенты!

<img src="./pytorch_logo.png">

- Удобный фреймворк для построения и обучения нейросетей
- Реализует динамический граф вычислений
- Вы просто пишете функцию, которая вычисляет нужное вам выражение, вычисляете (вызываете) её в некоторой точке (с некоторым аргументом) и можете получить градиенты в этой точке бесплатно!
- Специальные инструменты для построения нейросетей: модуль с большинством популярных слоёв, для которых градиенты уже реализованы
- Пока плохо поддерживает операции с разреженными матрицами

## GPU vs. CPU

- CPU – мощный вычислитель, но мало памяти доступно здесь и сейчас (см. лекцию про иерархию памяти)
- GPU – ОЧЕНЬ много ядер для выполнения простых операций
- CPU распределяет задачи по ядрам, GPU - по кластерам ядер

Только использование GPU (или [TPU](https://habr.com/ru/post/422317/)) позволит за разумное время обучить сложные модели для задач компьютерного зрения, NLP, RL и других областей.

### Какие операции нужны для работы нейросетей?

- Умножение матрицы на вектор и векторное сложение - полносвязный слой
- Поэлементые операции - нелинейности
- Локальные операции типа [MaxPooling](https://deepai.org/machine-learning-glossary-and-terms/max-pooling)
- Получение из вектора или матрицы скаляра

### Пример реализации операции матричного умножения

- Пример исходного кода, реализующего матричное умножение на CUDA, доступен [тут](https://www.quantstart.com/articles/Matrix-Matrix-Multiplication-on-the-GPU-with-Nvidia-CUDA)
- Сравнение с помощью PyTorch доступно по ссылке ниже [![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1aLpXukb3p-hzENDVaqBHltjRFfwpJ6f6)

## Резюме

- В основе нейросетей как и других вычислительных технологий лежит линейная алгебра
- Современные фреймворки (PyTorch, etc) позволяют вычислять градиенты автоматически
- GPU быстрее CPU для выполнения большого числа простых операций
- TPU ещё быстрее, но менее доступны
- [Colab](https://colab.research.google.com/) позволяет экспериментировать с этими технологиями удалённо